## 필요한 라이브러리 import

In [19]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

## 데이터 준비 

In [20]:
# 예시 데이터 로드 (실제 경로를 사용자의 데이터 파일 경로로 바꿔야 함)
df = pd.read_excel('./data/문자클래스데이터.xlsx')

df = df[["Message Body", "label"]]

df

,Message Body,label
0,"[Web발신]\n[KB국민카드]장우진님 08/14 카드대금 561,229원(08/0...",납부 예정
1,"[Web발신]\n[KB국민카드]장우진님 09/13 결제금액 20,000원(연회비 2...",납부 예정
2,"[Web발신]\n[KB국민카드]장우진님 09/14 결제금액 599,554원(연회비 ...",납부 예정
3,"[Web발신]\n[KB국민카드]장우진님 10/16 카드대금 642,364원(09/2...",납부 예정
4,"[Web발신]\n[KB국민카드]장우진님 11/14 카드대금 696,667원(11/0...",납부 예정
...,...,...
304,"[Web발신] MG체크카드(1874) 우*하님 12,500원승인(계좌잔액224,23...",결제 승인
305,"[Web발신] MG체크카드(1874) 우*하님 3,960원승인(계좌잔액236,739...",결제 승인
306,"[Web발신] MG체크카드(1874) 우*하님 40,000원승인(계좌잔액184,23...",결제 승인
307,"[Web발신] SWING 언락패스 이용권(12,900원)이 1일 후 KB국민카드70...",납부 예정


### 라벨 인코딩

In [21]:
label_to_index = {
    '납부 예정': 0,
    '결제 거절': 1,
    '결제 승인': 2,
    '자동 이체': 3,
    '미납': 4,
    '결제 취소': 5,
    '계좌 개설': 6
}

df['label_index'] = df['label'].map(label_to_index)

### 텍스트 데이터 벡터화

In [22]:
# 토크나이저 설정
max_words = 5000
max_len = 50
tokenizer = Tokenizer(num_words=max_words, lower=True)

# 데이터 준비 및 토크나이저 학습
tokenizer.fit_on_texts(df['Message Body'])

# 텍스트를 시퀀스로 변환하고 패딩 적용
sequences = tokenizer.texts_to_sequences(df['Message Body'])
text_data = pad_sequences(sequences, maxlen=max_len)

# 토크나이저 저장
tokenizer_json = tokenizer.to_json()
with open('./tokenizer/LSTM_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [23]:
# 라벨을 카테고리형으로 변환
labels = to_categorical(df['label_index']
                        ,num_classes=df['label'].nunique())

### 학습 데이터 분할

In [24]:
X_train, X_test, y_train, y_test = train_test_split(text_data, labels, test_size=0.3, random_state=42)


### 모델 구성

In [25]:
max_words = 10000  # 예시 값, 실제 데이터에 맞게 조정해야 함
max_len = 100      # 예시 값, 실제 데이터에 맞게 조정해야 함

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2))
model.add(Dense(df['label_index'].nunique(), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


### 모델 학습

In [26]:
batch_size = 32
model.fit(X_train, y_train, epochs=12, batch_size=batch_size, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.2738 - loss: 1.9311 - val_accuracy: 0.2727 - val_loss: 1.8531
Epoch 2/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3815 - loss: 1.7829 - val_accuracy: 0.3636 - val_loss: 1.7761
Epoch 3/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4411 - loss: 1.6250 - val_accuracy: 0.5909 - val_loss: 1.6452
Epoch 4/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6439 - loss: 1.4646 - val_accuracy: 0.3636 - val_loss: 1.5751
Epoch 5/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4815 - loss: 1.3930 - val_accuracy: 0.5909 - val_loss: 1.3856
Epoch 6/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7174 - loss: 1.1840 - val_accuracy: 0.5909 - val_loss: 1.2010
Epoch 7/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.6890 - loss: 1.0229 - val_accuracy: 0.6364 - val_loss: 1.0227
Epoch 8/12
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6467 - loss: 0.9032 - val_accuracy: 0.6364 - val_loss: 0.9027


### 학습된 모델 테스트

In [27]:
index_to_label = {
    0: '납부 예정',
    1: '결제 거절',
    2: '결제 승인',
    3: '자동 이체',
    4: '미납',
    5: '결제 취소',
    6: '계좌 개설'
}

def get_label_from_index(index):
    return index_to_label.get(index, "Unknown")

In [28]:
def predict_class(text):
    # 텍스트를 시퀀스로 변환
    sequences = tokenizer.texts_to_sequences([text])
    
    # 패딩 적용 (max_len은 실제 모델 학습에 사용된 시퀀스 길이에 맞춰야 함)
    padded_sequences = pad_sequences(sequences, maxlen=100)
    
    # 모델을 사용한 예측 수행
    prediction = model.predict(padded_sequences)
    
    # 가장 높은 확률을 가진 클래스의 인덱스를 찾음
    predicted_class = np.argmax(prediction, axis=1)
    
    return get_label_from_index(predicted_class[0])

In [29]:
# 새로운 텍스트에 대한 클래스 예측
new_text = """[Web발신]
[KB국민카드]장우진님 11/14 카드대금 696,667원(11/03기준) https://m.kbcard.com/z/a1
https://media.hermes.kt.com/data/MEDIA/messagebase/411701d13bde41a3bfe47f311258e345.LT-201105161103520-fgrl.png."""
predicted_class = predict_class(new_text)
print(f"Predicted class for the new text: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Predicted class for the new text: 납부 예정


### 학습된 모델 저장

In [30]:
if not os.path.exists('./model'):
    os.makedirs('./model')
model.save('./model/LSTM_model.h5')